In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import chess
import chess.pgn

/Users/ar-yukoh.shimizu/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('Using CUDA device:', device)
else:
    device = torch.device("cpu")
    print('CUDA is not available, using CPU')

CUDA is not available, using CPU


In [4]:
from chess_engine.utils.utils import uci_dict, uci_table, piece_values
from chess_engine.utils.dataloader import DataLoader, TestLoader
from chess_engine.model.policy import PolicyNetwork

ModuleNotFoundError: No module named 'chess_engine'

In [3]:
# Define the value network
class ValueNetwork(nn.Module):
    def __init__(self):
        super(ValueNetwork, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv3 = nn.Conv2d(128, 128, 3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU(inplace=True)
        self.fc1 = nn.Linear(128 * 8 * 8, 1024)
        self.bn4 = nn.BatchNorm1d(1024)
        self.relu4 = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(1024, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.fc2(x)
        x = torch.tanh(x)
        return x

# Define the policy network
policy_network = PolicyNetwork()
value_network = ValueNetwork()

# Define the loss functions
policy_criterion = nn.CrossEntropyLoss()
value_criterion = nn.MSELoss()

# Define the optimizer
policy_optimizer = optim.Adam(policy_network.parameters(), lr=0.01)
value_optimizer = optim.Adam(value_network.parameters(), lr=0.01)

# Define the training loop
def train_policy_value_networks(data):
    # Train the policy network using supervised learning
    policy_network.train()
    policy_optimizer.zero_grad()
    states = data['states']
    actions = data['actions']
    states_tensor = torch.tensor(states, dtype=torch.float32).unsqueeze(1)
    actions_tensor = torch.tensor(actions, dtype=torch.long)
    logits = policy_network(states_tensor)
    policy_loss = policy_criterion(logits.view(-1, 4096), actions_tensor)
    policy_loss.backward()
    policy_optimizer.step()

    # Train the value network using reinforcement learning
    value_network.train()
    value_optimizer.zero_grad()
    states = data['states']
    values = data['values']
    states_tensor = torch.tensor(states, dtype=torch.float32).unsqueeze(1)
    values_tensor = torch.tensor(values, dtype=torch.float32)
    outputs = value_network(states_tensor).view(-1)
    value_loss = value_criterion(outputs, values_tensor)
    value_loss.backward()
    value_optimizer.step()

# Train the networks using a combination of supervised and reinforcement learning
for i in range(num_iterations):
    data = generate_data_using_equivalent_of_AlphaGo_Fan()
    train_policy_value_networks(data)

NameError: name 'num_iterations' is not defined

In [ ]:
class RLAgent:
    def __init__(self, env, alpha=0.001, gamma=0.99):
        self.env = env
        self.alpha = alpha
        self.gamma = gamma
        self.value_network = ValueNetwork()
        self.optimizer = optim.Adam(self.value_network.parameters(), lr=self.alpha)
        self.model = MyNeuralNet()

    def get_action(self, state):
        move = self.model.predict(state)
        return move
    
    def play_game(self, num_games=100, temperature=1.0):
        data = []
        
        for i in range(num_games):
            board = chess.Board()
            states, policies, values = [], [], []
                
            while not board.is_game_over():
                # get the current state as input to the policy network
                state = createStateObj(board)
                states.append(state)
                
                # use the policy network to choose a move
                legal_moves = board.legal_moves
                legal_mask = torch.zeros(1968).to(device)
                for move in legal_moves:
                    legal_mask[uci_table.index(move.uci())] = 1
                
                policy = self.policy_network(state, legal_mask)
                policies.append(policy)
                
                # sample from the policy with a temperature parameter
                action_probs = self.softmax(policy / temperature)
                action = np.random.choice(len(action_probs), p=action_probs)
                
                # make the selected move on the board
                board.push(chess.Move.from_uci(uci_table[action]))
                
                # add the value of the current board to the list of values
                values.append(self.get_value(board))
            
            # update the data with the game results
            result = self.get_result(board)
            for j in range(len(states)):
                data.append((states[j], policies[j], values[j], result))

        return data

    def train(self, num_games):
        for i in range(num_games):
            self.play_game()
